In [1]:
import math
import pandas as pd
import numpy as np
from collections import defaultdict

In [2]:
seqs = pd.read_csv("./proteins.csv")
seqs.drop(["Unnamed: 0"],axis = 1,inplace = True)
print("Number of Seqence:",seqs.shape[0],"\nLength of Seqence:",seqs.shape[1])
seqs = np.array(seqs).tolist()

Number of Seqence: 200 
Length of Seqence: 100


In [3]:
elements=['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V',
          'W','Y']
dict_map = { ele:i for i, ele in enumerate(elements)}
inverse_dict_map = { i:ele for i, ele in enumerate(elements)}

In [4]:
def log(x):
    x=np.array(x)
    x[x<0]=0
    return  np.log(x+1e-200)

def get_p(beta):
    m=len(elements)
    p=np.full((m, m), beta)
    np.fill_diagonal(p, 4)
    p=p/np.sum(p,axis=1)
    return p

def compute_pr_theta_1(seqi,j,W,theta1):
    subsequnce=seqi[j:j+W]
   
    res=0
    for i,ele in enumerate(subsequnce):
        index=dict_map[ele]
        res+=theta1[index,i]
    return res

def compute_pr_theta_1_dp(seqs,prev_pr_list,current_p,prev_p,i,j,W):
    
    minus=prev_p[dict_map[seqs[i][j]],0]
    add=current_p[dict_map[seqs[i][j+W]],W-1]
    return prev_pr_list[i][j]-minus+add

def prepare_pos(seqs,pr_list_padded,W):
    pos=(np.arange(0,len(pr_list_padded)),np.argsort(-pr_list_padded, axis=1)[:,0])
    return pos

def compute_nc(seqs):
    nc=np.zeros(len(dict_map.keys()))
    for seq in seqs:
        for ele in seq:
            nc[dict_map[ele]]+=1
    return nc

def get_theta_1_in_test(seqs,sequence_ids,start_list,W,epsilon=1):
     
    #motifs= np.array([list(seqs[i][start:start+W])for i,start in enumerate(start_list) if i !=k])
    motifs= np.array([list(seqs[seq_id][start_list[i]:start_list[i]+W])for i,seq_id in enumerate(sequence_ids)])

    theta_1 = np.zeros((len(elements), W))
    for motif_pos in range(W):
        for i in range(len(elements)):
            
            theta_1[i,motif_pos] = np.sum(motifs[:,motif_pos] == elements[i])
       
        theta_1[ :,motif_pos] = (theta_1[:,motif_pos] + epsilon) / (len(motifs)+ 4 * epsilon)
    return log(theta_1)

def get_indicator_z(seqs,pos):
    z_list= [[0 for i in range(len(seq))] for seq in seqs]

    n_pos=len(pos[0])
    for i in range(n_pos):
        seq_id=pos[0][i]
        start_id=pos[1][i]
        z_list[seq_id][start_id]=1
    return z_list

def compute_likelihood(seqs,W,theta_0,theta_1,sequence_ids,start_list,lambda_dict):
    res=0
    for i,seq in enumerate(seqs):
        res+=log(lambda_dict[i])
        res+=get_pr_given_theta(seq,start_list[i], theta_0,theta_1,W)
    return res
def compute_joint_likelihood(seqs,W,theta_0,theta_1,z_list,lambda_value):
    #start_list=np.array([np.argmax(i) for i in z_list])
    res=0
    for i,seq in enumerate(seqs):
        res+=log(lambda_value[i])
        for j in range(len(seqs[i])-W+1):               
            res+=z_list[i][j]*get_pr_given_theta(seq,j, theta_0,theta_1,W)
    return res

def init_theta_0(seqs):
    theta_0=np.zeros(len(dict_map.keys()))
    total=sum([len(seq) for seq in seqs])
    for seq in seqs:
        for ele in seq:
            theta_0[dict_map[ele]]+=1
    return log(theta_0/total),theta_0

def init_theta_1(subsequence,p):
    # assert sum([s+k<len(sequences[i]) for i,s in enumerate(starts)])==0
   # inverse_dict_map = {i: ele for i, ele in enumerate(['A', 'C', 'G', 'T'])}
    index=[dict_map[ele] for ele in subsequence]
    #p[index]
    return  log(p[:,index])

def get_lambda_list(seqs,W):
    lambda_list=[]
    #mean
    mm=sum([len(seq)-W+1 for seq in seqs])/len(seqs)
    n=len(seqs)
    lambda_dict={}
    for i,seq in enumerate(seqs):
        m=len(seq)-W+1
        lambda_=1/m
        lambda_dict[i]=lambda_
    lambda_list.append(lambda_dict)
    return lambda_list
def get_theta_different(prev,cur):
    if prev is None:
        return float('inf')
    return np.sum(np.abs(np.exp(prev)-np.exp(cur)))

In [5]:
def get_pr_V(U,W):
    V=[[0  for _ in range(len(Ui)-W+1)]for Ui in U]
    for i in range(len(U)):
        for j in range(len(U[i])-W+1):
            temp_list= U[i][j:j+W]
            V[i][j]=min(temp_list)
    
    return V    

def get_pr_given_theta(sequence, s, theta_0,theta_1,W):

    t = len(sequence)
    res = 0
    for i in range(t):
        if i>=s and i<=s+W-1:
            res += theta_1[dict_map[sequence[i]],i - s]
        else:
            res += theta_0[dict_map[sequence[i]]]
        # print(theta_0[dict_map[sequence[i]]])
    return res

def sumLogProb(a, b):
    if a > b:
        return a + np.log1p(math.exp(b - a))
    else:
        return b + np.log1p(math.exp(a - b))
    
def sumlogProbList(l):
    # likelihood = logsumexp([hl, hh, lh, lh])
    likelihood = -math.inf
    for i in l:
        likelihood = sumLogProb(likelihood, i)

    return likelihood

In [6]:
def Estep(seqs,W,theta_0,theta_1):
        pr_list=[]      
        for i,seq in enumerate(seqs):
                temp=[]
                for j in range(0,len(seq)-W+1):
                    temp.append(get_pr_given_theta(seq, j, theta_0,theta_1,W))
                pr_list.append(temp)
        z_expected=[list(np.exp(p_sub-sumlogProbList(p_sub))) for p_sub in pr_list]
        return z_expected

def Mstep(seqs,W,z,nc,epsilon=0.25):
    nck=[[0] * 20 for i in range(W+1)]
    for k in range(1,W+1):
         for i,seq in enumerate(seqs):
            for j in range(0,len(seq)-W+1):
                indice=dict_map[seq[j+k-1]]
                nck[k][indice]+=z[i][j]  
        
    nck[0]=list(nc-np.sum(nck,axis=0))
    for i in range(len(nck[0])):
        nck[0][i]=max(0,nck[0][i])
    
    nck=np.array(nck)
    nck+=epsilon
    theta=nck/np.sum(nck,axis=1).reshape(-1,1)

    theta_0=log(theta[0])
    theta_1=log(theta[1:].T)

    return theta_0,theta_1

In [7]:
def test(seqs,W,lambda_list,p,theta_0,epsilon=1,quick_test=True):
    
    max_theta1_from_test=[None]*len(lambda_list)
    max_likelihood=[-float('inf')]*len(lambda_list)
    max_z=[None]*len(lambda_list)
    max_subsequence=[None]*len(lambda_list)
    set_motif={}
    prev_p=[None]*len(lambda_list)
    prev_pr_list=[None]*len(lambda_list)
    n=len(seqs)
   
    for k,outer_seq in enumerate(seqs):
        
        if quick_test and k==1:
            break
        for l in range(len(outer_seq)-W+1):
            #map sequence Xkl to theta 1
            subseq=outer_seq[l:l+W]
            theta_1=init_theta_1(subseq,p)
            
            #compute Pr(Xij/theta1)
            pr_list=[]
            for i,seq in enumerate(seqs):
                    temp=[compute_pr_theta_1(seq,0,W,theta_1)]
                    for j in range(1,len(seq)-W+1):
                        if l==0:
                            #brute force compute  pr(xij/theta1)
                            temp.append(compute_pr_theta_1(seq, j,W,theta_1))
                        else:
                            #use dp to compute pr(xij/theta1)
                            temp.append(compute_pr_theta_1_dp(seqs,prev_pr_list,theta_1,prev_p,i,j-1,W))
                    pr_list.append(temp)
            prev_p=theta_1 
            prev_pr_list=pr_list
      
            
            #padd the array into same length
            pad= max([ len(i) for i in prev_pr_list])
            
        
            pr_list_padded=np.array([i + [-float('inf')]*(pad-len(i)) for i in prev_pr_list])
            
            #pos is tuple (np.array of sequence id , np.array of pos index)
            pos=prepare_pos(seqs,pr_list_padded,W)
                
            for lambi,lambda_dict in enumerate(lambda_list):
                    topn=int(n*lambda_dict[0]*(len(seqs[0])-W+1))
                    dtopn=len(seqs)

                    seq_id_sub=pos[0][:topn]
                    start_list=pos[1][:topn]
                    #print(seq_id_sub,start_list)
                    theta_1=get_theta_1_in_test(seqs,seq_id_sub,start_list,W ,epsilon=epsilon) 
                    
                    likelihood=compute_likelihood(seqs,W,theta_0,theta_1,seq_id_sub,start_list,lambda_dict)
                   
                    if max_likelihood[lambi]<likelihood:
                        
                      
                        max_likelihood[lambi]=likelihood
                        #max_subsequence[lambi]=subseq
                        max_theta1_from_test[lambi]=theta_1
                        max_pos=tuple([seq_id_sub,start_list])
                        max_z[lambi]=get_indicator_z(seqs,max_pos)
    return max_theta1_from_test,max_z

In [8]:
def EM(seqs,W,theta_0_from_test,theta_1_from_test,U,epsilon,lambda_value):
    nc=compute_nc(seqs)
    prev=None
    threshold=0.000001
    t=0
    V=get_pr_V(U,W)
    theta_0=theta_0_from_test
    theta_1=theta_1_from_test
    for kk in range(200):
        t+=1
        
        z_expected=Estep(seqs,W,theta_0,theta_1)
        z_expected=[[z_expected[i][j]*V[i][j] for j in range(len(V[i]))] for i in range(len(V))]
        theta_0,theta_1=Mstep(seqs,W,z_expected,nc,epsilon=epsilon)
        likelihood=compute_joint_likelihood(seqs,W,theta_0,theta_1,z_expected,lambda_value)
#         print("t: ",t ,"State:",get_theta_different(prev,theta_1)<threshold)
        if  get_theta_different(prev,theta_1)<threshold:
            break
        prev=theta_1
        
    return  likelihood,z_expected,theta_1,theta_0,t

In [9]:
def get_consensus_from_p(x):
    return "".join(np.array(elements)[np.argmax(x,axis=0)])
def get_target_z(seqs,z_list,W):
    n=len(seqs)
    pos=prepare_pos(seqs,z_list,W)
    topn=n
    seq_id_sub=pos[0][:topn]
    start_list=pos[1][:topn]
    max_pos=tuple([seq_id_sub,start_list])           
    max_z=get_indicator_z(seqs,max_pos)
    return max_z,max_pos

def update_U(U,Z,W):
    for i in range(len(U)):
        for j in range(len(U[i])):
            temp_list= Z[i][max(j-W+1,0):j+1]
            U[i][j]=U[i][j]*(1-max(temp_list))
    return U

In [10]:
npass = 2
theta_1_list=[[] for j in range(npass)]
converged_motif=[[] for j in range(npass)]
tartget_motifs_list=[[] for j in range(npass)]
target_pos_list=[[] for j in range(npass)]

In [11]:
W = 10
beta = 0.15
p = get_p(beta)
quick_test = True
tract_test_dict=defaultdict(list)
U= [[1 for i in range(len(seq))]for seq in seqs]

In [12]:
def get_motif_seqs(seqs,z_list,W):
    motifs=[]
    for i,z in enumerate(z_list):
        if sum(z)==0:
            temp=[]
            motifs.append(temp)
        else:
            temp=[]
            index_list=np.where(z)[0]
            for k in index_list:
                temp.append(seqs[i][k:k+W])
            motifs.append(temp)
    return motifs

In [13]:
theta_0_from_test,cnt_dict=init_theta_0(seqs)
lambda_list = get_lambda_list(seqs,W)
max_theta1_from_test,max_z=test(seqs,W,lambda_list,p,theta_0_from_test,beta,quick_test=quick_test)
test_dict=[max_theta1_from_test,lambda_list,max_z]
for ipass in range(npass):
    for lambda_i,lambda_value in enumerate(lambda_list):
        theta_1_from_test=test_dict[0][lambda_i] 
        likelihood,z_list,theta_1,theta_0,total_iters=EM(seqs,W,theta_0_from_test,theta_1_from_test,U,beta,lambda_value) 
        tract_test_dict[0].append([lambda_value,get_consensus_from_p(theta_1_from_test),get_consensus_from_p(theta_1),total_iters]) 
        pad= max([len(i) for i in z_list])
        pr_list_padded=np.array([i + [-float('inf')]*(pad-len(i)) for i in z_list])
        target_z,target_pos=get_target_z(seqs,pr_list_padded,W)
        tartget_motifs=get_motif_seqs(seqs,target_z,W)
    theta_1_list[ipass]=theta_1
    target_pos_list[ipass] = target_pos
    converged_motif[ipass]=tract_test_dict
    tartget_motifs_list[ipass]=tartget_motifs
    U=update_U(U,z_list,W)

In [14]:
print("The position of the motif:\n", target_pos_list[0][1])
print("The motif of each sequence:\n", tartget_motifs_list[0])
print("The final motif:",tract_test_dict[0][0][2])

The position of the motif:
 [40 23  0 57  8 73 49 85 29 46 82  4 49 79 71 23 78 41  0 43 69 82 76  3
 25  5 36 82 34 74 30 89 15 54 42 49 84 66 71 22 23 11 51 31 77 45 30 83
 48  7 80 45 60 33 84 82 86 23 79 22 21 12 68 69 81 38 68 64 89 65 47 58
 53 12 56 30 12 72 16 85 57 42 86 81 30  8  6 65 32 82 24 53 11 26 15 75
 44 41 56  7 27 47 34 63 46  9 32 35 13 27 88 78 86 76 26 26 74 44 51  7
 29 87 46 64 64 72 13  5 57 19 80 10 68  1 28 64 33 82 56 88 63 41 57 31
 30 29  8 51 26 29 31 58 69 84 47  1 55 88 34 31 65 66  7  7 89 76 60 58
 37 73 84 87 18  6 18 14 83  1 60 16 68 42 31 10 47  8 57 32 52 51 32 59
 64  2 72 26 66 58  7  5]
The motif of each sequence:
 [[['V', 'S', 'C', 'S', 'V', 'L', 'T', 'C', 'G', 'P']], [['R', 'S', 'C', 'P', 'V', 'L', 'T', 'C', 'G', 'P']], [['Y', 'S', 'C', 'M', 'I', 'L', 'T', 'C', 'G', 'P']], [['A', 'S', 'C', 'P', 'V', 'L', 'T', 'C', 'G', 'P']], [['A', 'S', 'C', 'P', 'V', 'L', 'T', 'C', 'G', 'P']], [['A', 'K', 'C', 'P', 'V', 'L', 'T', 'C', 'G', 'P']], [['A', '

In [15]:
print("The position of the motif after:\n", target_pos_list[1][1])
# print("The motif of each sequence after:\n", tartget_motifs_list[1])
print("The final motif after:",tract_test_dict[0][1][2])

The position of the motif after:
 [83 83 72 84 50 35 66 60 76 19 62 65 82 21 20 46 57 70 30  4 36 47 45 81
 67 61 70 26 16 33 15 21  5 28 62 37 68 25 86 47 42 59 25 80 38 17  4 13
 37 23 25 16 86 12 11 37 21 61  9 86 59 72 22 15 59 75 11 76  8  6 35 87
 43 57 76 13 22 86 45  7 45 86 33 40 76 58 84 82  5 18 80 41 44 68  2 43
 90 54 78 31 12 36  7  8  8 84 71  4  2 65 58 21 53  1 78 60 54 86 21 60
 10 77 60  7 40 82 32 35 19 87 50 32 58 19 84 44 58 25  3 65 84 70 37 65
 54 62 37 23 59 89 45 41 54 22  8 85 13 68 55  4 20 82 44 72 38 13  5 76
 14 53 69 17 41 17 49 62 40 18 87  4 52 81  7 68 78 44 35 85 39 70 53 69
 18 44 42 11 22  1 66 50]
The final motif after: IWFFMEEARD
